Работа с временными рядами.
    
Известно ежемесячное число смертей в результате несчастного случая в США с января 1973 по декабрь 1978, необходимо построить прогноз на следующие 2 года. Ожидаю в конце уидеть график.
При выполнении работ напишите введение и вывод.Посторайтесь коментировать то что делаете. 

V1
08.02.24
Anatoly

In [54]:
import numpy as np
import pandas as pd


file_path_test ="C:/Users/qiarr/Desktop/task/lesson 21/accidental-deaths-in-usa-monthly.csv"
data = pd.read_csv(file_path_test, encoding='cp1251', )

In [55]:
data.head()

,Month,"Accidental deaths in USA: monthly, 1973 ? 1978"
0,1973-01,9007
1,1973-02,8106
2,1973-03,8928
3,1973-04,9137
4,1973-05,10017


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   Month                                           72 non-null     object
 1   Accidental deaths in USA: monthly, 1973 ? 1978  72 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [57]:
data['Date'] = pd.to_datetime(data['Month'], format='mixed')

data['Month'] = data['Date'].dt.month.astype(dtype=np.float64)
data['Year'] = data['Date'].dt.year.astype(dtype=np.float64)
x = data[['Month', 'Year']]
y = data["Accidental deaths in USA: monthly, 1973 ? 1978"]

In [58]:
data.head()

,Month,"Accidental deaths in USA: monthly, 1973 ? 1978",Date,Year
0,1.0,9007,1973-01-01,1973.0
1,2.0,8106,1973-02-01,1973.0
2,3.0,8928,1973-03-01,1973.0
3,4.0,9137,1973-04-01,1973.0
4,5.0,10017,1973-05-01,1973.0


In [59]:
x.head()


,Month,Year
0,1.0,1973.0
1,2.0,1973.0
2,3.0,1973.0
3,4.0,1973.0
4,5.0,1973.0


In [60]:
y.head()

0     9007
1     8106
2     8928
3     9137
4    10017
Name: Accidental deaths in USA: monthly, 1973 ? 1978, dtype: int64

In [61]:
from statsmodels.tsa.arima.model import ARIMA

In [62]:
# Возьмем последние 10 значений для тестирования
test_data = y[-10:]

# Оставим остальные данные для обучения
train_data = y[:-10]

учитывая что даных в принипе не так много а как мы возьмем период в 10 месяцев

In [63]:
model = ARIMA(train_data, order=(5,1,0)) # Подберите подходящие параметры (p,d,q) для вашего временного ряда
model_fit = model.fit()

# Прогнозирование
forecast = model_fit.forecast(steps=10) # Прогноз для следующих 10 периодов


In [64]:
print(forecast)

62    6915.704963
63    6731.563893
64    6986.742754
65    6989.623176
66    6927.383082
67    6942.215903
68    6879.595306
69    6913.034119
70    6913.207687
71    6912.177023
Name: predicted_mean, dtype: float64


In [65]:
test_data

62     7791
63     8129
64     9115
65     9434
66    10484
67     9827
68     9110
69     9070
70     8633
71     9240
Name: Accidental deaths in USA: monthly, 1973 ? 1978, dtype: int64

In [66]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(test_data, forecast)
print('MSE:', mse)

MSE: 5219858.154648555


In [67]:
mae = mean_absolute_error(test_data, forecast)
print('MAE:', mae)


MAE: 2172.175209322529


метрики в данном случае получились уж слишуом большими

In [68]:
from pmdarima import auto_arima

# Подбор оптимальных параметров
model = auto_arima(train_data, start_p=1, start_q=1,
                    max_p=3, max_q=3, m=12,
                    start_P=0, seasonal=True,
                    d=1, D=1, trace=True,
                    error_action='ignore',  
                    suppress_warnings=True, 
                    stepwise=True)

# Обучение модели с найденными параметрами
# Прогнозирование
forecast1 = model.predict(n_periods=10)  # Прогноз для следующих 10 периодов

print(forecast1)

Performing stepwise search to minimize aic


 ARIMA(1,1,1)(0,1,1)[12]             : AIC=716.458, Time=0.25 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=729.432, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=718.958, Time=0.12 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=714.515, Time=0.20 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=722.833, Time=0.03 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=716.487, Time=0.31 sec
 ARIMA(0,1,1)(0,1,2)[12]             : AIC=716.471, Time=0.45 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=716.971, Time=0.11 sec
 ARIMA(0,1,1)(1,1,2)[12]             : AIC=inf, Time=0.65 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=inf, Time=0.07 sec
 ARIMA(0,1,2)(0,1,1)[12]             : AIC=716.444, Time=0.23 sec
 ARIMA(1,1,0)(0,1,1)[12]             : AIC=716.098, Time=0.15 sec
 ARIMA(1,1,2)(0,1,1)[12]             : AIC=inf, Time=0.37 sec
 ARIMA(0,1,1)(0,1,1)[12] intercept   : AIC=716.261, Time=0.33 sec

Best model:  ARIMA(0,1,1)(0,1,1)[12]          
Total fit time: 3.276 seconds
62     770

In [69]:

mse = mean_squared_error(test_data, forecast1)
print('MSE:', mse)
mae = mean_absolute_error(test_data, forecast1)
print('MAE:', mae)

MSE: 260706.49038660462
MAE: 442.8320225021627


MAE довольно мала так что результат более чем удовлетворительный

Поиск настроек для ARIMA показал хорошие результаты ,меня заинтересовал способ найти лучшуу метрику MAE для данного датасета с помощью mamba поэтому возможно я обновлю задание позже с использованием данного способа 